In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re

In [2]:
data=pd.read_csv("/content/drive/MyDrive/sentiment-data.csv", encoding="latin-1")

In [3]:
data.head()

,customer_query,sentiment
0,Absolutely wonderful - silky and sexy and comf...,Positive
1,Love this dress! it's sooo pretty. i happene...,Positive
2,"I love, love, love this jumpsuit. it's fun, fl...",Positive
3,This shirt is very flattering to all due to th...,Positive
4,I aded this in my basket at hte last mintue to...,Positive


In [4]:
one_hot = pd.get_dummies(data['sentiment'])
data.drop(['sentiment'], axis=1, inplace=True)
data = pd.concat([data, one_hot], axis=1)
data.head()

,customer_query,Negative,Neutral,Positive
0,Absolutely wonderful - silky and sexy and comf...,0,0,1
1,Love this dress! it's sooo pretty. i happene...,0,0,1
2,"I love, love, love this jumpsuit. it's fun, fl...",0,0,1
3,This shirt is very flattering to all due to th...,0,0,1
4,I aded this in my basket at hte last mintue to...,0,0,1


In [5]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [6]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # removing contractions
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "that is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [8]:
data["customer_query"] = data["customer_query"].apply(preprocess_sentence)

In [16]:
X = data["customer_query"].values
y = data.drop(["customer_query"], axis=1).values

max_features = 20000
maxlen = 150
vocab_size = 20000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=150)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(21054, 150) (21054, 3)
(9024, 150) (9024, 3)


In [17]:
import pickle

# Save the tokenizer to a file using pickle
with open('/content/drive/MyDrive/tokenizer2.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
print(y)

[[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [19]:
embed_dim = 100
num_heads = 4
ff_dim = 200

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [20]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test)
)

Epoch 1/10
165/165 [==============================] - 370s 2s/step - loss: 1.0983 - accuracy: 0.3565 - val_loss: 1.0827 - val_accuracy: 0.3771
Epoch 2/10
165/165 [==============================] - 401s 2s/step - loss: 0.8030 - accuracy: 0.6097 - val_loss: 0.6527 - val_accuracy: 0.7029
Epoch 3/10
165/165 [==============================] - 405s 2s/step - loss: 0.5394 - accuracy: 0.7785 - val_loss: 0.5396 - val_accuracy: 0.7965
Epoch 4/10
165/165 [==============================] - 406s 2s/step - loss: 0.3670 - accuracy: 0.8709 - val_loss: 0.4653 - val_accuracy: 0.8343
Epoch 5/10
165/165 [==============================] - 413s 3s/step - loss: 0.2734 - accuracy: 0.9120 - val_loss: 0.4253 - val_accuracy: 0.8595
Epoch 6/10
165/165 [==============================] - 412s 3s/step - loss: 0.2078 - accuracy: 0.9337 - val_loss: 0.4424 - val_accuracy: 0.8652
Epoch 7/10
165/165 [==============================] - 421s 3s/step - loss: 0.1635 - accuracy: 0.9488 - val_loss: 0.4789 - val_accuracy: 0.8676

In [21]:
model.save("/content/drive/MyDrive/SentimentAnalysisTransformer.h5")

In [25]:
text = ["The product quality is not good."]

text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=150, dtype='int32', value=0)
sentiment = model.predict(text,batch_size=1,verbose = 2)[0]

print(sentiment)

if (np.argmax(sentiment) == 0):
  print("Negative")
elif (np.argmax(sentiment) == 1):
  print("Neutral")
else:
  print("Positive")

1/1 - 0s - 30ms/epoch - 30ms/step
[0.9324416  0.05212056 0.01543789]
Negative
